In [24]:
# ! pip install optuna
# ! pip install torch
# ! pip install pandas
# ! pip install numpy
# ! pip install matplotlib
# ! pip install kagglehub
# ! pip install scikit-learn

In [25]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

import optuna

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [27]:
# path = "/kaggle/input/fashionmnist"

import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fashionmnist


In [28]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
sample_df = df.sample(n=60000, random_state=42)

In [30]:
X = sample_df.iloc[:, 1:].values
y = sample_df.iloc[:, :1].values


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
st_scaler = StandardScaler()

In [33]:

X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

# Transformation

In [34]:
from torchvision.transforms import transforms

In [35]:
custom_transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR), # interpolation
    transforms.CenterCrop(224),
    transforms.ToTensor(), #converts PIL python image library to a Tensor & scales the pizel value between (0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # mean and standard deviation values 
                                                                                # are already present at pytorch page
])

# Dataset Class

In [36]:
# create custom dataset class
from PIL import Image
import numpy as np

class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels, transform):
        
        self.features = features
        self.labels = labels
        self.transform = transform
        

    def __len__(self):
        return len(self.features)


    def __getitem__(self, idx):
        
        # resize to (28, 28)
        image = self.features[idx].reshape(28, 28)


        # change datatype to np.uint8
        image = image.astype(np.uint8)

        # change Grayscale to color
        image = np.stack([image]*3, axis= -1)

        # convert array to PIL Image
        image = Image.fromarray(image)

        # apply transformation
        image = self.transform(image)

        # return
        return image, torch.tensor(self.labels[idx], dtype=torch.long)

In [37]:
train_dataset = FashionMNISTDataset(X_train, y_train, transform = custom_transform)
test_dataset = FashionMNISTDataset(X_test, y_test, transform = custom_transform)

In [38]:
# loading data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)


In [39]:
# fetch pre trained model
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [41]:
for param in vgg16.features.parameters():
    param.require_grad=False


In [42]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    
    nn.Linear(512, 10)
)

In [43]:
vgg16 = vgg16.to(device)

In [44]:
learning_rate = 0.005
epochs = 5


loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)




In [45]:
total_epoch_loss = []

for epoch in range(epochs):
    print(f"----------- Epoch {epoch+1}/{epochs} -----------")

    for batch_features, batch_labels in train_loader:
        
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        
        # forward pass
        outputs = vgg16(batch_features)

        # loss calculation
        loss = loss_function(outputs, batch_labels.squeeze())
        total_epoch_loss.append(loss.item())

        # backprop

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    print(f" avg epoch loss: {sum(total_epoch_loss)/len(total_epoch_loss)} ")

----------- Epoch 1/5 -----------
 avg epoch loss: 0.8810410915116469 
----------- Epoch 2/5 -----------
 avg epoch loss: 0.8162720378488302 
----------- Epoch 3/5 -----------
 avg epoch loss: 0.7811063271330463 
----------- Epoch 4/5 -----------
 avg epoch loss: 0.7605100051711003 
----------- Epoch 5/5 -----------
 avg epoch loss: 0.7414489341715972 


In [46]:
        
# evaluation

vgg16.eval()


# return accuracy

total = 0
correct = 0 
i = 1
cnt =0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:

        cnt +=1
        
        # move data to gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            
        test_outputs = vgg16(batch_features)

        _, predicted = torch.max(test_outputs, 1)
        
        
        
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels.squeeze()).sum().item()


accuracy = 100 * correct / total

print(accuracy) # 91.075

# 79.60833333333333 on :
# learning_rate = 0.005 & epochs = 5 
# classifier:     nn.Linear(25088, 1024),
#     nn.ReLU(),
#     nn.Dropout(0.2),

#     nn.Linear(1024, 512),
#     nn.ReLU(),
#     nn.Dropout(0.2),
    
#     nn.Linear(512, 10)


79.60833333333333
